In [1]:
from pandas import read_html
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib as plt
import sklearn
from bs4 import BeautifulSoup
import urllib.request

In [2]:
#Pulling regular season team ratings from 1986 to 2017
def team_ratings ():
    reg_team_rating_dframe = DataFrame()
    for x in range(1986,2017): #appends the rest of the dataframes
        url = 'https://www.basketball-reference.com/leagues/NBA_%d_ratings.html' %x
        dframe_list = pd.io.html.read_html(url)
        df = dframe_list[0]
        df.columns = df.columns.droplevel() #drops the unnecessary hierachy
        df['Year'] = Series ([x for i in range(len(df.index))])
        reg_team_rating_dframe = reg_team_rating_dframe.append(df)
    
    
    #2017 season doesnt have the hierachy like the rest of the seasons, append separately
    url1 = 'https://www.basketball-reference.com/leagues/NBA_2017_ratings.html'
    dframe_list1 = pd.io.html.read_html(url1)
    dframe17 = dframe_list1[0]
    dframe17['Year'] = Series ([2017 for i in range(len(dframe17.index))]) #year column
    reg_team_rating_dframe = reg_team_rating_dframe.append(dframe17)
    
    reg_team_rating_dframe = reg_team_rating_dframe.rename({"Rk":"Rank"},axis='columns')    #Feature Transformation
    #reg_team_rating_dframe.to_csv("Regular_Season_Team_Rating.csv",index=False)
    return reg_team_rating_dframe

In [3]:
#Find the champion of each season
def Champion():
    Champions_df = DataFrame(columns = ['Year','Team'])

    for x in range (1986,2018):
        html = 'https://www.basketball-reference.com/playoffs/NBA_%d.html' %x
        r = urllib.request.urlopen(html).read()
        soup = BeautifulSoup(r,'lxml')
        list1=[]
        for text in soup.find_all('p'):
            list1.append(text)
        element = list1[2]
        index = str(element).find(".html\">") 
        index = index + 7 #the starting index of the team
        Team = str(element)[index:str(element).find("</a><")] #substring "Team" is the team name
        series = Series (data = [x,Team],index = ['Year','Team'])
        Champions_df = Champions_df.append(series,ignore_index=True)
    return Champions_df


In [10]:
#Dataframe that details which round the team got eliminated
# 1 = Champion
# 2 = Runner's up
# 3 = Lost in the 3rd Round
# 4 = Lost in the 2nd Round
# 5 = Lost in the 1st Round
def playoff_elim ():
    playoff_elim_df = DataFrame()
    for x in range (1986,2018):
        url1 = 'https://www.basketball-reference.com/playoffs/NBA_%d.html' %x
        dframe_list = pd.io.html.read_html(url1)
        raw_dframe = dframe_list[16]
        teams_not_in_finals = raw_dframe['Team'][2:16]
        finalists = raw_dframe['Team'][0:2]
        df = DataFrame()
        positions = Series([0,1,2,3,3,4,4,4,4,5,5,5,5,5,5,5,5])
        df['Position'] = positions
        championsindex = int(x-1986)
        championteam = Champion_df.iloc[championsindex]['Team']
        teams = Champion_df.iloc[championsindex]
        finalists = finalists.loc[finalists != championteam]
        teams = teams.append(finalists,ignore_index=True)
        teams = teams.append(teams_not_in_finals,ignore_index=True)
        teams = teams.drop(index = [0])
        df['Teams'] = teams
        df['Year'] = Series(x,teams.index)
        df = df.drop(index= [0])
        playoff_elim_df = playoff_elim_df.append(df,ignore_index=True)
    return playoff_elim_df

In [5]:
teamratings_df = team_ratings()
teamratings_df.head()

,Rank,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,Year
0,1,Boston Celtics,E,A,67,15,0.817,9.41,112.40,103.16,9.24,9.06,112.54,103.67,8.87,1986
1,2,Milwaukee Bucks,E,C,57,25,0.695,9.04,112.05,103.09,8.95,8.69,112.17,103.58,8.59,1986
2,3,Los Angeles Lakers,W,P,62,20,0.756,7.74,113.69,106.38,7.32,6.84,113.18,106.72,6.47,1986
3,4,Atlanta Hawks,E,C,50,32,0.610,2.37,108.43,106.06,2.36,2.59,108.76,106.19,2.57,1986
4,5,Philadelphia 76ers,E,A,54,28,0.659,2.35,109.01,106.81,2.21,2.47,109.38,107.07,2.30,1986


In [12]:
teamratings_df.tail()

,Rank,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,Year
25,26,Phoenix Suns,W,P,24,58,0.293,-5.63,107.32,112.79,-5.47,-5.14,107.43,112.41,-4.98,2017
26,27,Philadelphia 76ers,E,A,28,54,0.341,-5.70,103.96,109.81,-5.85,-5.83,103.92,109.91,-5.99,2017
27,28,Los Angeles Lakers,W,P,26,56,0.317,-6.88,106.69,113.72,-7.02,-6.29,106.96,113.38,-6.42,2017
28,29,Brooklyn Nets,E,A,20,62,0.244,-6.73,104.61,111.34,-6.73,-6.74,104.63,111.37,-6.74,2017
29,30,Orlando Magic,E,SE,29,53,0.354,-6.57,104.23,111.11,-6.88,-6.61,104.30,111.21,-6.92,2017


In [6]:
Champion_df = Champion()
Champion_df.head()

,Year,Team
0,1986,Boston Celtics
1,1987,Los Angeles Lakers
2,1988,Los Angeles Lakers
3,1989,Detroit Pistons
4,1990,Detroit Pistons


In [11]:
playoff_elim_df = playoff_elim()
playoff_elim_df.head()

,Position,Teams,Year
0,1,Boston Celtics,1986.0
1,2,Houston Rockets,1986.0
2,3,Los Angeles Lakers,1986.0
3,3,Milwaukee Bucks,1986.0
4,4,Philadelphia 76ers,1986.0
